In [1]:
import numpy as np
import torch
from torch import nn, optim, Tensor
import torch.utils.data as Data
import os
import sys
sys.path.append('../')
import utils.csv as csv
import utils.validation as val
from models.transformer import TransformerClassifier

d:\LenovoSoftstore\Anaconda\envs\yolov5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
outputs = torch.rand([3, 4, 1])
labels = torch.rand(32)
outputs = outputs.view([4, -1])
softmax = nn.Softmax(dim=1)
outputs = softmax(outputs)
outputs.size(1)
# _, predicted = torch.max(outputs.data, 1)
# predicted

3

In [17]:
# file path
PATH='D:\\Deutschland\\FUB\\master_thesis\\data\\gee\\output'
DATA_DIR = os.path.join(PATH, 'monthly_mean')
LABEL_CSV = 'label.csv'
label_path = os.path.join(PATH, LABEL_CSV)

In [18]:
# general hyperparameters
BATCH_SIZE = 128
LR = 0.01
EPOCH = 100
SEED = 2048

In [19]:
# hyperparameters for LSTM
input_size = 5
hidden_size = 20
num_layers = 1
num_classes = 5
layer1_dim = 256
layer2_dim = 128

In [20]:
def numpy_to_tensor(x_data:np.ndarray, y_data:np.ndarray):
    # embedding
    embedding = nn.Embedding(8000, input_size)
    # reduce dimention from (n, 1) to (n, )
    y_data = y_data.reshape(-1)
    x_set = torch.from_numpy(x_data)
    y_set = torch.from_numpy(y_data)
    x_set = embedding(x_set).detach()
    return x_set, y_set

In [21]:
def build_dataloader(x_set:Tensor, y_set:Tensor, batch_size:int, seed:int):
    # dataset = Data.TensorDataset(x_set, y_set)
    # # split dataset
    # size = len(dataset)
    # train_size, val_size = round(0.8 * size), round(0.2 * size)
    # generator = torch.Generator().manual_seed(seed)
    # train_dataset, val_dataset = Data.random_split(dataset, [train_size, val_size], generator)
    x_train = x_set[:444]
    y_train = y_set[:444]
    x_val = x_set[444:]
    y_val = y_set[444:]
    train_dataset = Data.TensorDataset(x_train, y_train)
    val_dataset = Data.TensorDataset(x_val, y_val)
    # data_loader
    train_loader = Data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=2)
    val_loader = Data.DataLoader(val_dataset,batch_size=len(val_dataset), shuffle=True,num_workers=2)
    return train_loader, val_loader

In [22]:
def train(model:nn.Module, epoch:int):
    total_step = len(train_loader)
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        # forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # record training loss
        if i % total_step == 0:
            print('Epoch[{}/{}],Step[{}/{}],Loss:{:.4f}'
            .format(epoch+1,EPOCH,i+total_step,total_step,loss.item()))

In [23]:
def validate(model:nn.Module):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for (values, labels) in val_loader:
            values = values.to(device)
            labels = labels.to(device)
            outputs = model(values)
            total += labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

In [24]:
if __name__ == "__main__":
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # dataset
    x_data, y_data = csv.to_numpy(DATA_DIR, label_path)
    x_set, y_set = numpy_to_tensor(x_data, y_data)
    train_loader, val_loader = build_dataloader(x_set, y_set, BATCH_SIZE, SEED)
    # model
    model = ClassificationLSTM(input_size, hidden_size, layer1_dim, layer2_dim, num_layers, num_classes).to(device)
    # loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), LR)
    # train and validate model
    for epoch in range(EPOCH):
        train(model, epoch)
        validate(model)
    # save model
    # torch.save(model, '../outputs/model.pkl')

Epoch[1/100],Step[4/4],Loss:1.6101
Test Accuracy of the model: 10.363247863247864 %
Epoch[2/100],Step[4/4],Loss:1.6074
Test Accuracy of the model: 17.628205128205128 %
Epoch[3/100],Step[4/4],Loss:1.6225
Test Accuracy of the model: 20.405982905982906 %
Epoch[4/100],Step[4/4],Loss:1.6027
Test Accuracy of the model: 39.31623931623932 %
Epoch[5/100],Step[4/4],Loss:1.5847
Test Accuracy of the model: 23.076923076923077 %
Epoch[6/100],Step[4/4],Loss:1.5626
Test Accuracy of the model: 22.863247863247864 %
Epoch[7/100],Step[4/4],Loss:1.5590
Test Accuracy of the model: 21.26068376068376 %
Epoch[8/100],Step[4/4],Loss:1.5168
Test Accuracy of the model: 20.61965811965812 %
Epoch[9/100],Step[4/4],Loss:1.4782
Test Accuracy of the model: 19.444444444444443 %
Epoch[10/100],Step[4/4],Loss:1.5031
Test Accuracy of the model: 20.833333333333332 %
Epoch[11/100],Step[4/4],Loss:1.4893
Test Accuracy of the model: 18.91025641025641 %
Epoch[12/100],Step[4/4],Loss:1.4344
Test Accuracy of the model: 20.72649572649

KeyboardInterrupt: 